In [1]:
from dataclasses import dataclass
from typing import List, Tuple

In [2]:
@dataclass
class TreeNode:
    val: int = 0
    left: "TreeNode | None" = None
    right: "TreeNode | None" = None

    def __str__(self):
        return f"TreeNode({self.val}) -> ({self.left}, {self.right})"

In [3]:
def dfs_recursive(node: TreeNode | None) -> None:
    if not node:
        return
    # Process the current node
    print(node.val)
    # Recur on the left subtree
    dfs_recursive(node.left)
    # Recur on the right subtree
    dfs_recursive(node.right)


# Example Usage
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
root.left.left = TreeNode(4)
root.left.right = TreeNode(5)
print(root)
dfs_recursive(root)

TreeNode(1) -> (TreeNode(2) -> (TreeNode(4) -> (None, None), TreeNode(5) -> (None, None)), TreeNode(3) -> (None, None))
1
2
4
5
3


In [4]:
def dfs_iterative(root: TreeNode | None) -> None:
    if not root:
        return
    stack = [root]
    while stack:
        node = stack.pop()
        # Process the current node
        print(node.val)
        # Push right child first so that left child is processed first
        if node.right:
            stack.append(node.right)
        if node.left:
            stack.append(node.left)


# Example Usage
dfs_iterative(root)

1
2
4
5
3


In [8]:
def dfs_recursive(
    graph: dict[str, list[str]], node: str, visited: set[str] | None = None
):
    if not node:
        return
    if visited is None:
        visited = set()
    if node not in visited:
        print(node)
        visited.add(node)
        for neighbor in graph[node]:
            dfs_recursive(graph, neighbor, visited)


# Example Usage
graph = {
    "A": ["B", "C"],
    "B": ["A", "D", "E"],
    "C": ["A", "F"],
    "D": ["B"],
    "E": ["B", "F"],
    "F": ["C", "E"],
}

dfs_recursive(graph, "A")

A
B
D
E
F
C


In [33]:
def dfs_iterative(graph: dict[str, list[str]], start: str):
    if not graph or not start or start not in graph:
        return
    visited: set[str] = set()
    stack = [start]
    while stack:
        node = stack.pop()
        if node not in visited:
            print(node)
            visited.add(node)
            for neighbor in graph[node]:
                if neighbor not in visited:
                    stack.append(neighbor)


# Example Usage
dfs_iterative(graph, "A")

A
C
F
E
B
D


In [12]:
graph: dict[str, list[str]] = {
    "shirt": ["tie", "belt"],
    "tie": ["jacket"],
    "belt": ["jacket"],
    "watch": [],
    "underpants": ["pants", "shoes"],
    "pants": ["belt", "shoes"],
    "socks": ["shoes"],
    "shoes": [],
    "jacket": [],
}

In [48]:
from dataclasses import dataclass, field


@dataclass
class NodeTimes:
    entry: int = -1
    exit: int = -1


@dataclass
class TimeStamp:
    time: int = 0


def dfs_iterative_colored_adj_list(
    graph: dict[str, list[str]],
    start: str,
    color: dict[str, str],
    times: dict[str, NodeTimes],
    timestamp: TimeStamp,
) -> dict[str, NodeTimes]:
    stack = [(start, "entry")]

    while stack:
        node, state = stack.pop()

        if state == "entry":
            if color[node] == "white":
                color[node] = "gray"
                times[node].entry = timestamp.time
                timestamp.time += 1

                stack.append((node, "exit"))

                for neighbor in reversed(graph[node]):
                    if color[neighbor] == "white":
                        stack.append((neighbor, "entry"))

        elif state == "exit":
            color[node] = "black"
            times[node].exit = timestamp.time
            timestamp.time += 1

    return times

# graph_adj_list: dict[str, list[str]] = {
#     "shirt": ["tie", "belt"],
#     "tie": ["jacket"],
#     "belt": ["jacket"],
#     "watch": [],
#     "underpants": ["pants", "shoes"],
#     "pants": ["belt", "shoes"],
#     "socks": ["shoes"],
#     "shoes": [],
#     "jacket": [],
# }

graph_adj_list = {
    "A": ["B", "C"],
    "B": ["D", "E"],
    "C": ["F"],
    "D": ["B"],
    "E": ["B", "F"],
    "F": ["C", "E"],
}
# Initialize all nodes as white (unvisited) and create a dictionary to store entry/exit times
color = {node: "white" for node in graph_adj_list}
times = {node: NodeTimes(-1, -1) for node in graph_adj_list}
timestamp = TimeStamp()

times_adj_list = dfs_iterative_colored_adj_list(
    graph=graph_adj_list, start="C", color=color, times=times, timestamp=timestamp
)
print("Entry and Exit Times (Adjacency List):")
for node, times in sorted(times_adj_list.items(), key=lambda x: x[1].exit, reverse=True):
    print(f"Node {node}: Entry = {times.entry}, Exit = {times.exit}")

Entry and Exit Times (Adjacency List):
Node C: Entry = 0, Exit = 9
Node F: Entry = 1, Exit = 8
Node E: Entry = 2, Exit = 7
Node B: Entry = 3, Exit = 6
Node D: Entry = 4, Exit = 5
Node A: Entry = -1, Exit = -1


In [25]:
from typing import NamedTuple

NodeTimes = NamedTuple("NodeTimes", [("entry", int), ("exit", int)])


def dfs_iterative_colored_adj_list(
    graph: dict[str, list[str]],
    start: str,
    color: dict[str, str],
    times: dict[str, NodeTimes],
    timestamp: int,
):
    stack = [(start, "entry")]  # Stack to hold nodes and state ('entry' or 'exit')
    while stack:
        node, state = stack.pop()

        if state == "entry":
            if color[node] == "white":
                # Node is being visited, mark it as gray and set entry time
                color[node] = "gray"
                times[node] = times[node]._replace(entry=timestamp)
                timestamp += 1

                # Push the node back with 'exit' state
                stack.append((node, "exit"))

                # Add all unvisited neighbors to the stack
                for neighbor in reversed(graph[node]):
                    if color[neighbor] == "white":
                        stack.append((neighbor, "entry"))

        elif state == "exit":
            # Node has been processed, mark it as black and set exit time
            color[node] = "black"
            times[node] = times[node]._replace(exit=timestamp)
            timestamp += 1

    return timestamp


def topological_sort(graph: dict[str, list[str]]):

    # Initialize all nodes as white (unvisited) and create a dictionary to store entry/exit times
    color = {node: "white" for node in graph}
    times = {node: NodeTimes(-1, -1) for node in graph}
    timestamp = 0

    # Perform DFS from each unvisited node
    for i in graph:
        if color[i] == "white":
            timestamp = dfs_iterative_colored_adj_list(
                graph=graph, start=i, color=color, times=times, timestamp=timestamp
            )

    # Sort vertices by exit time in descending order
    sorted_vertices = sorted(
        [x for x in graph], key=lambda x: times[x].exit, reverse=True
    )

    return sorted_vertices


# # Example Usage for Adjacency List
graph_adj_list = {
    "A": ["B", "C"],
    "B": ["A", "D", "E"],
    "C": ["A", "F"],
    "D": ["B"],
    "E": ["B", "F"],
    "F": ["C", "E"],
}

graph_adj_list: dict[str, list[str]] = {
    "shirt": ["tie", "belt"],
    "tie": ["jacket"],
    "belt": ["jacket"],
    "watch": [],
    "underpants": ["pants", "shoes"],
    "pants": ["belt", "shoes"],
    "socks": ["shoes"],
    "shoes": [],
    "jacket": [],
}
topological_sort(graph_adj_list)
# times_adj_list = dfs_iterative_colored_adj_list(graph_adj_list, 'socks')
# print("Entry and Exit Times (Adjacency List):")
# for node, times in sorted(times_adj_list.items(), key = lambda x: x[1].entry):
#     print(f"Node {node}: Entry = {times.entry}, Exit = {times.exit}")

['socks',
 'underpants',
 'pants',
 'shoes',
 'watch',
 'shirt',
 'belt',
 'tie',
 'jacket']

In [29]:
def dfs(
    v: str,
    adj_list: dict[str, list[str]],
    visited: dict[str, bool],
    entry: dict[str, int],
    exit: dict[str, int],
    time: list[int],
):
    visited[v] = True
    entry[v] = time[0]
    time[0] += 1

    # Explore all the vertices adjacent to this vertex
    for neighbor in adj_list[v]:
        if not visited[neighbor]:
            dfs(neighbor, adj_list, visited, entry, exit, time)

    exit[v] = time[0]
    time[0] += 1


def topological_sort_using_times(adj_list: dict[str, list[str]]):
    visited = {v: False for v in adj_list}
    entry = {v: -1 for v in adj_list}
    exit = {v: -1 for v in adj_list}
    TIME = [0]  # To simulate a global timer

    # Perform DFS from each unvisited node
    for i in adj_list:
        if not visited[i]:
            dfs(i, adj_list, visited, entry, exit, TIME)

    # Sort vertices by exit time in descending order
    sorted_vertices = sorted([x for x in adj_list], key=lambda x: exit[x], reverse=True)

    return sorted_vertices


# Example usage
graph_adj_list: dict[str, list[str]] = {
    "shirt": ["tie", "belt"],
    "tie": ["jacket"],
    "belt": ["jacket"],
    "watch": [],
    "underpants": ["pants", "shoes"],
    "pants": ["belt", "shoes"],
    "socks": ["shoes"],
    "shoes": [],
    "jacket": [],
}

topological_order = topological_sort_using_times(graph_adj_list)
print(topological_order)

['socks', 'underpants', 'pants', 'shoes', 'watch', 'shirt', 'belt', 'tie', 'jacket']


In [18]:
times_adj_list

{'shirt': NodeTimes(entry=None, exit=None),
 'tie': NodeTimes(entry=None, exit=None),
 'belt': NodeTimes(entry=None, exit=None),
 'watch': NodeTimes(entry=None, exit=None),
 'underpants': NodeTimes(entry=None, exit=None),
 'pants': NodeTimes(entry=None, exit=None),
 'socks': NodeTimes(entry=0, exit=3),
 'shoes': NodeTimes(entry=1, exit=2),
 'jacket': NodeTimes(entry=None, exit=None)}

In [5]:
# dfs detect cycle using entry and exit times
def dfs_detect_cycle(graph: dict[str, list[str]], start: str):
    color = {node: "white" for node in graph}
    stack = [(start, "entry")]
    timestamp = 0

    while stack:
        node, state = stack.pop()

        if state == "entry":
            if color[node] == "white":
                color[node] = "gray"
                timestamp += 1
                stack.append((node, "exit"))
                for neighbor in reversed(graph[node]):
                    if color[neighbor] == "white":
                        stack.append((neighbor, "entry"))
                    elif color[neighbor] == "gray":
                        return True

        elif state == "exit":
            color[node] = "black"
            timestamp += 1

    return False


# Example Usage
graph_cycle = {"A": ["B"], "B": ["C"], "C": ["D"], "D": ["B"]}

print(dfs_detect_cycle(graph_cycle, "A"))  # Output: True


# dfs detect cycle using parent
def dfs_detect_cycle_parent(graph: dict[str, list[str]], start: str):
    color = {node: "white" for node in graph}
    parent = {node: None for node in graph}
    stack = [(start, "entry")]

    while stack:
        node, state = stack.pop()

        if state == "entry":
            if color[node] == "white":
                color[node] = "gray"
                stack.append((node, "exit"))
                # why reversed? because we want to visit neighbors in the order they were added
                for neighbor in reversed(graph[node]):
                    if color[neighbor] == "white":
                        stack.append((neighbor, "entry"))
                        parent[neighbor] = node
                    elif parent[node] != neighbor:
                        return True

        elif state == "exit":
            color[node] = "black"

    return False


# Example Usage
graph_cycle = {"A": ["B"], "B": ["C"], "C": ["D"], "D": ["B"]}

print(dfs_detect_cycle_parent(graph_cycle, "A"))  # Output: True

True
True


In [8]:
# dfs using hashset optionally detect cycle and return path
def dfs_detect_cycle_path(
    graph: dict[str, list[str]], start: str, detect_cycle: bool = False
):
    visited: set[str] = set()
    stack: list[tuple[str, list[str]]] = [(start, [])]

    while stack:
        node, path = stack.pop()

        if node in visited:
            if detect_cycle:
                return path + [node]
            continue

        visited.add(node)
        path.append(node)

        for neighbor in reversed(graph[node]):
            stack.append((neighbor, path.copy()))

    return None


# Example Usage
graph_cycle = {"A": ["B"], "B": ["C"], "C": ["D"], "D": ["B"]}

print(
    dfs_detect_cycle_path(graph_cycle, "A", detect_cycle=False)
)  # Output: ['A', 'B', 'C', 'D', 'B']

None


In [6]:
# dfs and also detect cycle using hashsset
def dfs_detect_cycle_set(graph: dict[str, list[str]], start: str):
    def dfs(node: str, parent: None | str) -> bool:
        if node in visited:
            return True
        visited.add(node)
        for neighbor in graph[node]:
            if neighbor != parent and dfs(neighbor, node):
                return True
        return False

    visited: set[str] = set()
    return dfs(start, None)


# Example Usage
graph_cycle = {"A": ["B"], "B": ["C"], "C": ["D"], "D": ["B"]}

print(dfs_detect_cycle_set(graph_cycle, "A"))  # Output: True

True


In [ ]:
# DFS(G, s)
# Input: A graph, G; a source vertex s
# Purpose: visits all vertices of G reachable from s

#     for each vertex v in Vertices(G)
#         color[v] = white
#     DFS_Visit(s)
# DFS_Visit(v)
# Input: the node currently being visited, v
# Postcondition: All nodes reachable from v will have been visited
#     color[v] = gray             // Mark v as discovered
#     for each u adjacent to v
#         if color[u] == white
#             DFS_Visit(u)        // Found a new node; explore it
#     color[v] = black            // Mark v as fully explored

In [105]:
def dfs_iterative_colored_adj_matrix(matrix, start):
    n = len(matrix)
    # Initialize all nodes as white (unvisited) and create a list to store entry/exit times
    color = ["white"] * n
    times = [NodeTimes(None, None) for _ in range(n)]
    stack = [(start, "entry")]  # Stack to hold nodes and state ('entry' or 'exit')
    timestamp = 0

    while stack:
        node, state = stack.pop()

        if state == "entry":
            if color[node] == "white":
                # Node is being visited, mark it as gray and set entry time
                color[node] = "gray"
                times[node] = times[node]._replace(entry=timestamp)
                timestamp += 1

                # Push the node back with 'exit' state
                stack.append((node, "exit"))

                # Add all unvisited neighbors to the stack
                for neighbor in range(n - 1, -1, -1):
                    if matrix[node][neighbor] == 1 and color[neighbor] == "white":
                        stack.append((neighbor, "entry"))

        elif state == "exit":
            # Node has been processed, mark it as black and set exit time
            color[node] = "black"
            times[node] = times[node]._replace(exit=timestamp)
            timestamp += 1

    return times


# Example Usage for Adjacency Matrix
# Graph:
# A - 0
# B - 1
# C - 2
# D - 3
# E - 4
# F - 5
matrix_adj = [
    [0, 1, 1, 0, 0, 0],  # A
    [1, 0, 0, 1, 1, 0],  # B
    [1, 0, 0, 0, 0, 1],  # C
    [0, 1, 0, 0, 0, 0],  # D
    [0, 1, 0, 0, 0, 1],  # E
    [0, 0, 1, 0, 1, 0],  # F
]

times_adj_matrix = dfs_iterative_colored_adj_matrix(matrix_adj, 0)
print("Entry and Exit Times (Adjacency Matrix):")
for i, times in enumerate(times_adj_matrix):
    print(f"Node {i}: Entry = {times.entry}, Exit = {times.exit}")

Entry and Exit Times (Adjacency Matrix):
Node 0: Entry = 0, Exit = 11
Node 1: Entry = 1, Exit = 10
Node 2: Entry = 6, Exit = 7
Node 3: Entry = 2, Exit = 3
Node 4: Entry = 4, Exit = 9
Node 5: Entry = 5, Exit = 8


In [104]:
def dfs_detect_cycle(graph: dict[str, list[str]], start: str):
    if not graph or not start or start not in graph:
        return None

    # Initialize all nodes as white (unvisited)
    color = {node: "white" for node in graph}
    # Initialize a dictionary to store entry and exit times
    times = {node: NodeTimes(None, None) for node in graph}
    # Initialize the stack with the start node
    stack = [start]
    timestamp = 0
    parent = {node: None for node in graph}  # To track the parent of each node

    while stack:
        node = stack[-1]

        if color[node] == "white":
            # Node is being visited, mark it as gray and set entry time
            color[node] = "gray"
            times[node] = NodeTimes(entry=timestamp, exit=None)
            timestamp += 1

            # Add all unvisited neighbors to the stack
            for neighbor in reversed(graph[node]):
                if color[neighbor] == "white":
                    parent[neighbor] = node
                    color[neighbor] = "gray"
                    stack.append(neighbor)
                elif color[neighbor] == "gray" and parent[node] != neighbor:
                    # Found a back edge, which indicates a cycle
                    print(f"Cycle detected: Back edge from {node} to {neighbor}")
            print(color)
            # print(stack)

        elif color[node] == "gray":
            print("Processing exit", node)
            # Node has been processed, mark it as black and set exit time
            color[node] = "black"

            times[node] = times[node]._replace(exit=timestamp)
            timestamp += 1
            stack.pop()  # Pop the node after processing

    return times


# Example Usage
# graph_adj_list = {
#     'A': ['B', 'C'],
#     'B': ['D', 'E'],
#     'C': ['F'],
#     'D': ['A'],  # Adding a back edge to create a cycle
#     'E': ['F'],
#     'F': []
# }
graph_adj_list = {
    "A": ["B", "C"],
    "B": ["A", "C"],
    "C": ["A", "B"],
    # 'D': ['B'],
    # 'E': ['B', 'F'],
    # 'F': ['C', 'E']
}


times_adj_list = dfs_detect_cycle(graph_adj_list, "A")
print("Entry and Exit Times (Adjacency List):")
for node, times in times_adj_list.items():
    print(f"Node {node}: Entry = {times.entry}, Exit = {times.exit}")

{'A': 'gray', 'B': 'gray', 'C': 'gray'}
Processing exit B
Processing exit C
Processing exit A
Entry and Exit Times (Adjacency List):
Node A: Entry = 0, Exit = 3
Node B: Entry = None, Exit = 1
Node C: Entry = None, Exit = 2


In [19]:
def dfs_detect_cycle(graph: dict[str, list[str]], start: str):
    if not graph or not start or start not in graph:
        return None

    # Initialize all nodes as white (unvisited)
    color = {node: "white" for node in graph}
    # Initialize a dictionary to store entry and exit times
    times = {node: NodeTimes(None, None) for node in graph}
    # Initialize the stack with the start node and its parent (None)
    stack = [(start, None)]
    timestamp = 0

    while stack:
        node, parent = stack.pop()

        if color[node] == "white":
            # Node is being visited, mark it as gray and set entry time
            color[node] = "gray"
            times[node] = NodeTimes(entry=timestamp, exit=None)
            timestamp += 1

            # Add all unvisited neighbors to the stack
            for neighbor in reversed(graph[node]):
                if color[neighbor] == "white":
                    stack.append((neighbor, node))
                elif color[neighbor] == "gray" and neighbor != parent:
                    # Found a back edge, which indicates a cycle
                    print(f"Cycle detected: Back edge from {node} to {neighbor}")

        elif color[node] == "gray":
            # Node has been processed, mark it as black and set exit time
            color[node] = "black"
            times[node] = times[node]._replace(exit=timestamp)
            timestamp += 1

    return times


# Example Usage
graph_adj_list = {"A": ["B", "C"], "B": ["A", "C"], "C": ["A", "B"]}

times_adj_list = dfs_detect_cycle(graph_adj_list, "A")
print("Entry and Exit Times (Adjacency List):")
for node, times in times_adj_list.items():
    print(f"Node {node}: Entry = {times.entry}, Exit = {times.exit}")

Cycle detected: Back edge from C to A
Entry and Exit Times (Adjacency List):
Node A: Entry = 0, Exit = None
Node B: Entry = 1, Exit = None
Node C: Entry = 2, Exit = 3


In [12]:
def find_not_smaller_elements(arr):
    if not arr:
        return []

    result = []
    max_so_far = float("-inf")

    for i in range(len(arr) - 1, -1, -1):
        if arr[i] >= max_so_far:
            result.append(arr[i])
            max_so_far = arr[i]

    result.reverse()  # Reverse to get the correct order
    return result


# Example cases
example1 = [16, 17, 4, 3, 5, 2]
example2 = [1, 2, 3, 4, 5]
example3 = [5, 4, 3, 2, 1]
example4 = [10, 20, 10, 30, 20, 40, 30]

print(find_not_smaller_elements(example1))  # Output: [17, 5, 2]
print(find_not_smaller_elements(example2))  # Output: [5]
print(find_not_smaller_elements(example3))  # Output: [5, 4, 3, 2, 1]
print(find_not_smaller_elements(example4))  # Output: [40, 30]

[17, 5, 2]
[5]
[5, 4, 3, 2, 1]
[40, 30]


In [1]:
from collections import defaultdict, deque


def is_bipartite(graph, n):
    color = [-1] * n
    for start in range(n):
        if color[start] == -1:
            queue = deque([start])
            color[start] = 0
            while queue:
                node = queue.popleft()
                for neighbor in graph[node]:
                    if color[neighbor] == -1:
                        color[neighbor] = 1 - color[node]
                        queue.append(neighbor)
                    elif color[neighbor] == color[node]:
                        return False, []
    return True, color


def minimize_gift_cost(n, m, relations, k):
    graph = defaultdict(list)
    for u, v in relations:
        graph[u].append(v)
        graph[v].append(u)

    bipartite, color = is_bipartite(graph, n)

    if not bipartite:
        return "The graph is not bipartite; no valid gifting solution exists."

    set1 = [i for i in range(n) if color[i] == 0]
    set2 = [i for i in range(n) if color[i] == 1]

    min_gift_count = min(len(set1), len(set2))
    return min_gift_count * k


# Example usage
n = 6  # Number of employees
m = 5  # Number of neighbouring relations
relations = [(0, 1), (1, 2), (1, 3), (3, 4), (4, 5)]
k = 100  # Cost of each gift

print(minimize_gift_cost(n, m, relations, k))  # Output: 300

200


In [2]:
from collections import defaultdict, deque


def find_gift_recipients(n, edges):
    # Initialize graph
    graph = defaultdict(list)
    for u, v in edges:
        graph[u].append(v)
        graph[v].append(u)

    visited = [False] * n
    recipients = set()

    for u in range(n):
        if not visited[u]:
            # Choose the current node as a recipient
            recipients.add(u)
            visited[u] = True
            for v in graph[u]:
                visited[v] = True

    # Ensure at least one employee receives a gift
    if not recipients:
        recipients.add(0)  # If no recipients selected, select the first employee

    return recipients


def minimize_gift_cost(n, m, relations, k):
    recipients = find_gift_recipients(n, relations)
    return len(recipients) * k


# Example usage
n = 6  # Number of employees
m = 5  # Number of directed relations
relations = [(1, 2), (2, 3), (1, 3), (4, 4), (4, 5)]
k = 100  # Cost of each gift

print(minimize_gift_cost(n, m, relations, k))  # Output: 100

400


In [4]:
from collections import defaultdict


def find_gift_recipients(n, adj_list):
    visited = [False] * n
    recipients = set()

    for u in range(n):
        if not visited[u]:
            # Choose the current node as a recipient
            recipients.add(u)
            visited[u] = True
            for v in adj_list[u]:
                visited[v] = True

    # # Ensure at least one employee receives a gift
    # if not recipients:
    #     recipients.add(0)  # If no recipients selected, select the first employee

    return recipients


def minimize_gift_cost(n, m, adj_list, k):
    recipients = find_gift_recipients(n, adj_list)
    return len(recipients) * k


# Example usage
n = 6  # Number of employees
m = 5  # Number of directed relations
adj_list = {0: [1], 1: [2, 3], 2: [], 3: [4], 4: [5], 5: []}
k = 100  # Cost of each gift

print(minimize_gift_cost(n, m, adj_list, k))  # Output depends on the recipients found

400


In [20]:
adj_list = {0: [1], 1: [2, 3], 2: [], 3: [4], 4: [5], 5: []}


def find_min_gift(adj_list: dict[int, list[int]]) -> int:
    visited: set[int] = set()
    gift_count = 0
    for employee in adj_list:
        if employee not in visited:
            gift_count += 1
            stack = [employee]
            while stack:
                current_employee = stack.pop()
                visited.add(current_employee)
                for neighbor in adj_list[current_employee]:
                    if neighbor not in visited:
                        stack.append(neighbor)
    return gift_count


print(find_min_gift(adj_list))

1
